Import required packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from copy import deepcopy
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle

/workspaces/recommendation-system/.conda/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Read the file

In [2]:
file = pd.read_excel('shortcourses2566.xlsx')

Count the number of courses

In [3]:
course_counts = pd.DataFrame(file)['หลักสูตรอบรมระยะสั้น'].value_counts()
courses = pd.Series(course_counts.index)
courses = courses.sort_values().set_axis(range(0,len(courses)))
number_of_courses = len(course_counts)

Create Series of users

In [4]:
s_name = file.loc[:, 'ชื่อ-นามสกุล (อังกฤษ)']
Users = pd.Series(s_name, name='User')
Users

0             PORPHAING JANTIP
1         THANAKORN DARASRISAK
2             KARNJANA EAMTANG
3             KARNJANA EAMTANG
4       THITIMUN VORATHONGCHAI
                 ...          
6122         KITTINON SANTASUP
6123      WITTAWAT SERMSRIPONG
6124               NON NAKKARA
6125         PHATTHARAPHON LIN
6126      THANAPAT SEEMAKAJOHN
Name: User, Length: 6127, dtype: object

Create Series of emails

In [5]:
s_email = file.loc[:, 'อีเมล'].fillna("")
Emails = pd.Series(s_email ,name='Email')
Emails

0            tanghaoren17@gamil.com
1               tuakung21@gmail.com
2            karnjana.aon@gmail.com
3            karnjana.aon@gmail.com
4          thitimun.rama1@gmail.com
                   ...             
6122           kittinon_s@cmu.ac.th
6123        wittawat_serm@cmu.ac.th
6124          non_nakkara@cmu.ac.th
6125    phattharaphon_lin@cmu.ac.th
6126         thanapat_see@cmu.ac.th
Name: Email, Length: 6127, dtype: object

Provide a score to each user based on their email domain

In [6]:
email_score = []
for data in Emails:
    if data != '':
        if data.split('@')[1] == 'cmu.ac.th':
            email_score.append(2)
        else:
            email_score.append(1)
    else:
        email_score.append(0)
email_score = pd.Series(email_score, name='Score Email')

email_score

0       1
1       1
2       1
3       1
4       1
       ..
6122    2
6123    2
6124    2
6125    2
6126    2
Name: Score Email, Length: 6127, dtype: int64

Email Score Statistic

In [7]:
zero_score_count = email_score.where(email_score == 0).count()
one_score_count = email_score.where(email_score == 1).count()
two_score_count = email_score.where(email_score == 2).count()

print("Number of students who filled cmu email:", two_score_count)
print("Number of students who filled other email:", one_score_count)
print("Number of students who did not fill email:", zero_score_count)

Number of students who filled cmu email: 1268
Number of students who filled other email: 4853
Number of students who did not fill email: 6


Create function to calculate age-education score

In [8]:
def getAgeEducationScore(age, limit_age):
    if age <= limit_age:
        score = 0
    elif age <= limit_age + 2:
        score = 1
    else:
        score = 2
    return score

Create set of the educational range

In [9]:
set_nan = {'อื่นๆ (-)', np.nan}
set_primaryschool = {'ประถมศึกษา', 'อื่นๆ (ป.4)', 'อื่นๆ (ป.7)', 'อื่นๆ (ป7)'}
set_middleschool = {'มัธยมศึกษาตอนต้น', 'Secondary school', 'อื่นๆ (มศ.3)'}
set_highschool = {'มัธยมศึกษาตอนปลาย', 'High school', 'Vocational', 'การศึกษานอกระบบ', 
                  'ประกาศนียบัตรวิชาชีพ (ปวช.)', 'อื่นๆ (ม.ปลาย จบหลักสูตรEMR เป็นจนท.ปฏิบัติการ)',
                  'อื่นๆ (กำลังศึกษาชั้นมัธยมศึกษาตอนปลาย)', 'อื่นๆ (กำลังศึกษาชั้นมัธยมศึกษาปีที่6)', 
                  'อื่นๆ (มศ.5)'}
set_bachelor = {'ปริญญาตรี', 'Bachelor degree', 'Diploma', 'High Vocational', 
                'ประกาศนียบัตรวิชาชีพชั้นสูง (ปวส.)', 'อื่นๆ (กำลังศึกษาในระดับปริญญาตรี)', 
                'อื่นๆ (กำลังศึกษาปริญญาตรี สาขารังสีเทคนิค)', 'อื่นๆ (ปริญญาแพทยศาสตร์บัณฑิต)', 
                'อื่นๆ (นักศึกษาแพทย์ปี 5)', 'อื่นๆ (นักศึกษาแพทย์ มช ปี4 ศูนย์เชียงราย)', 
                'อื่นๆ (แพทยศาสตร์บัณฑิต)', 'อื่นๆ (แพทย์)', 'อื่นๆ (ประกาศณียบัตรผู้ช่วยพยาบาล)', 
                'อนุปริญญา', 'อื่นๆ (ป.ตรี)', 'อื่นๆ (ผู้ช่วยพยาบาล)'}
set_masterdocter = {'ปริญญาโท', 'ปริญญาเอก', "Master's degree", 'Other (OBGYN specalist lavel 1)', 
                    'Other (Residency)', 'Ph.D.', 'อื่นๆ (Internal Medicine)', 
                    'อื่นๆ (เฉพาะทาง)', 'อื่นๆ (วุฒิบัตร)', 'อื่นๆ (วว.ออร์โธปิดิกส์)', 
                    'อื่นๆ (วุฒิบัตรแสดงความรู้ความชำนาญในการประกอบวิชาชีพเภสัชกรรม สาขาเภสัชบำบัด)', 
                    'อื่นๆ (วุฒิบัตรผู้เชี่ยวชาญสาขาทันตกรรมทั่วไป)', 'อื่นๆ (วุฒิบัตรศัลยศาสตร์และแม็กซิลโลเฟเชียล)'}

list_degree = ((set_nan, 0), (set_primaryschool, 16), (set_middleschool, 19), 
               (set_highschool, 22), (set_bachelor,26), (set_masterdocter,40))

Create Series of Age-Education

In [10]:
ages = file.loc[:, 'อายุ']
educations = file.loc[:, 'วุฒิการศึกษา']
age_education_scores = []

for i,x in enumerate(educations):
    for y in list_degree:
        if x in y[0]:
            age_education_scores.append(getAgeEducationScore(ages[i], y[1]))
            
age_education_scores = pd.Series(age_education_scores, name='Age Education Score')

Age-Education Score Statistic

In [11]:
zero_score_count = age_education_scores.where(age_education_scores == 0).count()
one_score_count = age_education_scores.where(age_education_scores == 1).count()
two_score_count = age_education_scores.where(age_education_scores == 2).count()

print("Number of students who are currently in the educational system:", zero_score_count)
print("Number of students who were recently graduated:", one_score_count)
print("Number of students who are not in the educational system:", two_score_count)

Number of students who are currently in the educational system: 4229
Number of students who were recently graduated: 487
Number of students who are not in the educational system: 1411


Create Series of status

In [12]:
status = file.loc[:, 'สถานะ'].fillna("")
status = pd.Series(status ,name='Status')
status

0       ชำระเงิน
1       ชำระเงิน
2       ชำระเงิน
3       ชำระเงิน
4       ชำระเงิน
          ...   
6122    ชำระเงิน
6123    ชำระเงิน
6124    ชำระเงิน
6125    ชำระเงิน
6126    ชำระเงิน
Name: Status, Length: 6127, dtype: object

Provide a score to each user based on their purchase status

In [13]:
payment_score = []
for x in status:
    if x == 'ชำระเงิน':
        payment_score.append(2)
    if x == 'ไม่ผ่านการอนุมัติ':
        payment_score.append(1)
    if x == 'ค้างชำระ':
        payment_score.append(0)
payment_score = pd.Series(payment_score)

Purchase Status Score Statistics

In [14]:
zero_score_count = payment_score.where(payment_score == 0).count()
one_score_count = payment_score.where(payment_score == 1).count()
two_score_count = payment_score.where(payment_score == 2).count()

print("Number of students who are in arrears:", zero_score_count)
print("Number of students whose payment was not approved:", one_score_count)
print("Number of students with payment approval:", two_score_count)

Number of students who are in arrears: 531
Number of students whose payment was not approved: 124
Number of students with payment approval: 5472


Create Series of address

In [15]:
address = file.loc[:, 'ที่อยู่'].fillna("")
address = pd.Series(address ,name='Status')
address

0                                                        
1       125/27 ซ.3 ถ.ราษฎรยินดี ต.หน้าเมือง อ.เมือง จ....
2                                                        
3                                                        
4                                                        
                              ...                        
6122                                                     
6123                                                     
6124                                                     
6125                                                     
6126                                                     
Name: Status, Length: 6127, dtype: object

Provide a score to each user based on whether they provide address information or not

In [16]:
address_score = [ 0 if x == '' else 1 for x in address]
address_score = pd.Series(address_score)

Address Score Statistic

In [17]:
zero_score_count = address_score.where(address_score == 0).count()
one_score_count = address_score.where(address_score == 1).count()

print("Number of students who did not fill address:", zero_score_count)
print("Number of students who filled address:", one_score_count)

Number of students who did not fill address: 4005
Number of students who filled address: 2122


Create Series of data

In [18]:
enrollment = file.loc[:, 'วันที่สมัครอบรม'].fillna("")
enrollment = pd.Series(enrollment ,name='Date')
enrollment = enrollment.str.slice(start=-8, stop=-6)

Provide a score to each user based on their enrollment time

In [19]:
enrollment_set = {'08','09','10','15'}
enrollment_score = [ 1 if x in enrollment_set else 0 for x in enrollment]
enrollment_score = pd.Series(enrollment_score)

Create DataFrame by merging these 4 Series and calculate impressive level

In [20]:
user = file.loc[:, 'ชื่อ-นามสกุล (อังกฤษ)']
course = file.loc[:, 'หลักสูตรอบรมระยะสั้น']
d = {
    'User': user,
    'Course': course,
    'Email Score': email_score,
    'Age Education Score': age_education_scores,
    'Enrollment': enrollment_score,
    'Payment Score': payment_score,
    'Address Score': address_score,
    'Score': 1 + email_score*0.375 + age_education_scores*0.25 + enrollment_score*0.5  + payment_score*1 + address_score*0.25
}
df = pd.DataFrame(d)

Create user-course table

In [21]:
# all user, course, score have the same length
data = {
    'User': df['User'],
    'Course': df['Course'],
    'Score': df['Score'],
}

predata = pd.DataFrame(data)


Calculate sparsity and csr matrix

In [22]:
# Pivot table by rotating course
data = pd.pivot_table(predata, values='Score', index='Course', columns='User').fillna(0)
# pd.set_option('display.max_rows', data.shape[0]+1)
# pd.set_option('display.max_columns', data.shape[0]+1)



Load tfidf_matrix

In [23]:
with open('tfidf_matrix.pickle', 'rb') as f:
    tfidf_matrix = pickle.load(f)
'''this will save 'tfidf_matrix' to a file named 'tfidf_matrix.pickle' in the same directory as 'tfidf.ipynb', 
and then load it from that file in 'knn.ipynb'.'''

"this will save 'tfidf_matrix' to a file named 'tfidf_matrix.pickle' in the same directory as 'tfidf.ipynb', \nand then load it from that file in 'knn.ipynb'."

Create Item-User rated matrix

In [24]:
knn_matrix = csr_matrix(data.values)

In [25]:
from scipy.sparse import hstack

combined_matrix = hstack([tfidf_matrix, knn_matrix])

Euclidean Distance & Cosine Similarity

In [26]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute').fit(knn_matrix)
model_combined = NearestNeighbors(metric='cosine', algorithm='brute').fit(combined_matrix)
'''This will create a new matrix where each row is the concatenation of the 
corresponding rows from tfidf_matrix and knn_matrix. The NearestNeighbors model
is then fit on this combined matrix.'''

'This will create a new matrix where each row is the concatenation of the \ncorresponding rows from tfidf_matrix and knn_matrix. The NearestNeighbors model\nis then fit on this combined matrix.'

Recommender function using KNN model

In [27]:
def recommender_knn(course_name):
    n_recommendations = int( number_of_courses ** (1/2) )
    idx = process.extractOne(course_name, courses)[2]
    distances, indices = model_knn.kneighbors(knn_matrix[idx], n_neighbors=n_recommendations+1, return_distance=True)
    recommendations = [courses[i].where(i!=idx) for i in indices]
    recommended_courses = recommendations[0][1:]
    course_distances = distances[0][1:]
    d = {
        'Course': recommended_courses,
        'Cosine Distance': course_distances
    }
    results = pd.DataFrame(data=d)
    n_distance = results['Cosine Distance'].where(results['Cosine Distance'] < 0.8).count()
    return results.head(n_distance)

In [28]:
recommender_knn('หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (Basic Principle of Diagnostic Radiology Imaging Instruments)')

,Course,Cosine Distance
152,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.672307
42,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.722699
43,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.754774


Recommender function using hybrid model

In [29]:
def recommender_hybrid(course_name):
    n_recommendations = int( number_of_courses ** (1/2) )
    idx = process.extractOne(course_name, courses)[2]
    distances, indices = model_combined.kneighbors(combined_matrix[idx], n_neighbors=n_recommendations+1, return_distance=True)
    recommendations = [courses[i].where(i!=idx) for i in indices]
    recommended_courses = recommendations[0][1:]
    course_distances = distances[0][1:]
    d = {
        'Course': recommended_courses,
        'Cosine Distance': course_distances
    }
    results = pd.DataFrame(data=d)
    n_distance = results['Cosine Distance'].where(results['Cosine Distance'] < 0.8).count()
    return results.head(n_distance)

In [30]:
recommender_hybrid('หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (Basic Principle of Diagnostic Radiology Imaging Instruments)')

,Course,Cosine Distance
152,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.672318
42,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.722705
43,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.754771


KNN Recommender function for all courses

In [31]:
def recommender_knn_all_courses(course_name):
    idx = process.extractOne(course_name, courses)[2]
    # print('Selected movie:', courses[idx], 'Index:', idx)
    distances, indices = model_knn.kneighbors(knn_matrix[idx], n_neighbors=len(courses))
    recommendations = [courses[i].where(i!=idx) for i in indices]
    recommended_courses = recommendations[0][1:]
    scores = 1 - distances
    course_distances = scores[0][1:]
    d = {
        'Course': recommended_courses,
        'Score': course_distances
    }
    results = pd.DataFrame(data=d)
    results = results.sort_index().rename_axis('Index')
    return results

KNN Recommender function using username

In [32]:
def recommender_knn_by_user(user_name, n_recommendations):
    df = {
        'User': pd.Series(file['ชื่อ-นามสกุล (อังกฤษ)']),
        'Course': pd.Series(file['หลักสูตรอบรมระยะสั้น'])
    }
    
    user_course = pd.DataFrame(df)
    selected_user_name = user_course.loc[user_course['User'] == user_name]
    selected_courses = selected_user_name['Course']
    
    recommended_courses = [ recommender_knn_all_courses(x) for x in selected_courses]
    
    # pre dataframe
    df = pd.DataFrame({
        'Course': [],
        'Score': []
    }).rename_axis('Index')
    
    for x in recommended_courses:
        df = df._append(x)
    df =  df.sort_values('Score', ascending=False).drop_duplicates('Course')
    return df.head(n_recommendations)

Hybrid Recommender function for all courses

In [33]:
def recommender_hybrid_all_courses(course_name):
    idx = process.extractOne(course_name, courses)[2]
    # print('Selected movie:', courses[idx], 'Index:', idx)
    distances, indices = model_combined.kneighbors(combined_matrix[idx], n_neighbors=len(courses))
    recommendations = [courses[i].where(i!=idx) for i in indices]
    recommended_courses = recommendations[0][1:]
    scores = 1 - distances
    course_distances = scores[0][1:]
    d = {
        'Course': recommended_courses,
        'Score': course_distances
    }
    results = pd.DataFrame(data=d)
    results = results.sort_index().rename_axis('Index')
    return results

Hybrid Recommender function using username

In [34]:
def recommender_hybrid_by_user(user_name, n_recommendations):
    df = {
        'User': pd.Series(file['ชื่อ-นามสกุล (อังกฤษ)']),
        'Course': pd.Series(file['หลักสูตรอบรมระยะสั้น'])
    }
    
    user_course = pd.DataFrame(df)
    selected_user_name = user_course.loc[user_course['User'] == user_name]
    selected_courses = selected_user_name['Course']
    
    recommended_courses = [ recommender_hybrid_all_courses(x) for x in selected_courses]
    
    # pre dataframe
    df = pd.DataFrame({
        'Course': [],
        'Score': []
    }).rename_axis('Index')
    
    for x in recommended_courses:
        df = df._append(x)
    df =  df.sort_values('Score', ascending=False).drop_duplicates('Course')
    return df.head(n_recommendations)

Items show permanence whereas, people change with time
Items are fewer in numbers to deal with. Which leads to smaller similarity matrix. Amazon and Netflix use it!
Better for New users:
— Him selecting just one item will let us provide recommendations
— But for user based, new user has to wait until next build of similarity matrix (which is the only computational part of the framework)

In [44]:
recommender_knn_by_user('THANAKORN DARASRISAK', 10)

,Course,Score
Index,,
43,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.753351
131,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.277301
110,รังสีวิทยาวินิจฉัย,0.219784
152,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.176883
124,สารสนเทศทางสาธารณสุข 2566 (Public Health Infor...,0.073328
41,การวิจัยแบบผสมผสานทางสุขภาพ (Mixed Methods Res...,0.032697
136,หลักสูตรอบรมเข้มข้นระยะสั้นแพทย์-วิศวกรรมและวิ...,0.022270
117,วิทยาศาสตร์การแพทย์คลินิก สาขาวิชาเวชศาสตร์ฉุก...,0.018849
98,ทักษะความเป็นนักดนตรี,0.017921


Training and Testing

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

Predata of training

In [37]:
# Take 2 columns from Table
course_score = df[['Course', 'Score']]

# Sort the course
course = df['Course'].sort_values().unique()

# Calculate the mean of each course
course_mean = course_score.groupby('Course')
course_mean = course_mean.mean().loc[:, 'Score']

# Take 2 columns from Table
course_user = df[['Course', 'User']]

# Count the number of users who enrolled each course 
course_count = course_user.value_counts('Course')
course_count = course_count.sort_index()

# Take feature columns from Table
course_feature = df[['Course', 'Email Score', 'Age Education Score', 'Enrollment', 'Payment Score', 'Address Score']]
agg_functions = {
    'Email Score': 'mean',
    'Age Education Score': 'mean',
    'Enrollment': 'mean',
    'Payment Score': 'mean',
    'Address Score': 'mean',
}
course_feature = course_feature.groupby('Course').aggregate(agg_functions)
course_score[course_score['Score'] > 5]

,Course,Score


Create the train dataframe

In [38]:
course_feature['Count'] = course_count
course_feature['Score'] = course_mean
train_df = pd.DataFrame(course_feature)
pd.set_option('display.max_rows', train_df.shape[0]+1)
filtered_train_df = train_df[train_df['Score'] > 4]

Split the train and the test

In [39]:
# Get X and y features variables
X = train_df[['Email Score', 'Age Education Score', 'Enrollment', 'Payment Score', 'Address Score', 'Count']]
y = train_df['Score']

# Use the train test split function
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=10, test_size=0.25
)

In [40]:
regressor = KNeighborsRegressor(n_neighbors=3)
regressor.fit(X_train, y_train)
predictions = regressor.predict(X_test)

In [41]:
print("Mean Squared Error:", mean_squared_error(predictions,y_test) / 4)
print("Mean Absolute Error:", mean_absolute_error(predictions,y_test) / 4)

Mean Squared Error: 0.07524013250199839
Mean Absolute Error: 0.10501315558555732
